In [ ]:
# Querying Yahoo Finance Data

In [1]:
import requests
import pandas as pd
import numpy as np

In [28]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

req = requests.get('http://query1.finance.yahoo.com/v7/finance/options/SPY?date=1642723200', headers = headers)

In [ ]:
req.content.decode()

In [48]:
df = pd.read_json(req.content.decode())

In [99]:
newdf = pd.DataFrame.from_dict(df.optionChain.result[0], orient='index')

In [151]:
newdf.iloc[5].values[0][:][0]

{'expirationDate': 1642723200,
 'hasMiniOptions': False,
 'calls': [{'contractSymbol': 'SPY220121C00115000',
   'strike': 115.0,
   'currency': 'USD',
   'lastPrice': 351.51,
   'change': 0.0,
   'percentChange': 0.0,
   'volume': 158,
   'openInterest': 0,
   'bid': 0.0,
   'ask': 0.0,
   'contractSize': 'REGULAR',
   'expiration': 1642723200,
   'lastTradeDate': 1639686287,
   'impliedVolatility': 1e-05,
   'inTheMoney': True},
  {'contractSymbol': 'SPY220121C00120000',
   'strike': 120.0,
   'currency': 'USD',
   'lastPrice': 346.97,
   'change': 0.0,
   'percentChange': 0.0,
   'volume': 25,
   'openInterest': 0,
   'bid': 0.0,
   'ask': 0.0,
   'contractSize': 'REGULAR',
   'expiration': 1642723200,
   'lastTradeDate': 1639683117,
   'impliedVolatility': 1e-05,
   'inTheMoney': True},
  {'contractSymbol': 'SPY220121C00125000',
   'strike': 125.0,
   'currency': 'USD',
   'lastPrice': 340.56,
   'change': 0.0,
   'percentChange': 0.0,
   'volume': 175,
   'openInterest': 0,
   'bid

In [146]:
newdf[0]

underlyingSymbol                                                  SPY
expirationDates     [1639699200, 1639958400, 1640131200, 164021760...
strikes             [115.0, 120.0, 125.0, 130.0, 135.0, 140.0, 145...
hasMiniOptions                                                  False
quote               {'language': 'en-US', 'region': 'US', 'quoteTy...
options             [{'expirationDate': 1642723200, 'hasMiniOption...
Name: 0, dtype: object